In [1]:
import sys
import sqlite3
import pandas as pd

sys.path.append('../bioreactor-technical-analysis')
#sys.path.append('/Users/armanrafian/Programs/thesis/bioreactor-technical-analysis/')

In [2]:
import biota.steadyfluxes as steady

## Run 2nd experiment

In [3]:
def runSim( showGraphs=False, **kwargs):
    
    impellerDiameter=kwargs['tankDiameter']/3
    initVol = 0.76 * kwargs['workingVolume']
    STR_Humbird_20kl = steady.Bioreactor(wv=kwargs['workingVolume'],t=kwargs['tankDiameter'],d=impellerDiameter,
                                         n=kwargs['rpm'],p_back=kwargs['backPressure'],u_s=kwargs['superficialVel'],
                                         mf_O2_gas=kwargs['moleFracO2'],mf_CO2_gas=kwargs['moleFracCO2'],v0=initVol,
                                         ns=kwargs['initCells'],Temp=kwargs['temp'],Np=kwargs['powerNumber'],
                                         rho=kwargs['mediumDensity'],mu=kwargs['mediumViscosity'],vvd=kwargs['vesselVolDay'],
                                         perfAMM=kwargs['perfAmmrate'],perfLAC=kwargs['perfLactateRate'])

    uptakeList = [kwargs['glutamineUptakeRate'], kwargs['glucoseUptakeRate'], kwargs['oxygenUptakeRate']]
    prodList = [kwargs['carbonDioxideProdRate'], kwargs['ammoniaProductionRate'], kwargs['lactateProductionRate']]
    limitsList = [kwargs['ammoniaLimit'], kwargs['lactateLimit'], kwargs['CO2Limit'], kwargs['turbLengthLimit']]
    cell_Humbird = steady.Cell(mu=kwargs['growthRate'],uptakes=uptakeList,prod=prodList,rho=kwargs['massDensity'],rad=kwargs['cellRadius'],wetmass=kwargs['wetmass'],dmf=kwargs['dryMassFraction'],limits=limitsList)
    
    bioreactor = STR_Humbird_20kl
    cell = cell_Humbird
    return steady.brute(count=kwargs['count'],b=bioreactor,c=cell,dbls=kwargs['doublings'],rpmlims=kwargs['rpmlims'],uslims=kwargs['supervellims'],nslims=kwargs['celldenslims'],graphs=showGraphs)

In [4]:
kwargs = {
    'workingVolume': 20000,
    'tankDiameter':  2.34,
    'rpm': 42.3,
    'backPressure': 1.3,
    'superficialVel': 0.004,
    'moleFracO2': 0.21,
    'moleFracCO2':  0.03,
    'initCells': 4e6,
    'temp': 310,
    'powerNumber': 5,
    'mediumDensity': 1000,
    'mediumViscosity': 9e-4,
    'vesselVolDay': 0.0,
    'perfLactateRate': 5.0,
    'perfAmmrate': 5.0,
    'growthRate': 0.029,
    'glutamineUptakeRate': 0,
    'glucoseUptakeRate': 0,
    'oxygenUptakeRate': 0.49, 
    'carbonDioxideProdRate': 0.593,
    'ammoniaProductionRate': 0.013571,
    'lactateProductionRate': 0.135707, 
    'massDensity': 1030,
    'cellRadius': 18e-6,
    'wetmass': 3000,
    'dryMassFraction': 0.3,
    'ammoniaLimit': 5,
    'lactateLimit': 50, 
    'CO2Limit': 100,
    'turbLengthLimit': 20e-6,
    'count': 50,
    'doublings': 7,
    'rpmlims': (1,75),
    'supervellims': (0.0001,0.005),
    'celldenslims': (5.9e5,6.1e5)
}

In [5]:
df = runSim(**kwargs)

In [6]:
df

,Constraint,Maximum Yield [g/L wet]
0,overall,47.632072
1,ammonia,47.632072
2,lactate,47.632072
3,CO2,131.708818
4,kla,63.668296
5,mixing,71.497455
6,hydrodynamic stress,386.966163
7,superficial velocity,332.760836


In [7]:
df_dict_list = df.to_dict(orient='records')
df_dict = {}
for item in df_dict_list:
    df_dict[item['Constraint']] = item['Maximum Yield [g/L wet]']
    
document = {
    "inputs": {},
    "outputs": {}
}

for key, value in kwargs.items():
    document['inputs'][key] = value

for key, value in df_dict.items():
    document['outputs'][key] = value

document

{'inputs': {'workingVolume': 20000,
  'tankDiameter': 2.34,
  'rpm': 42.3,
  'backPressure': 1.3,
  'superficialVel': 0.004,
  'moleFracO2': 0.21,
  'moleFracCO2': 0.03,
  'initCells': 4000000.0,
  'temp': 310,
  'powerNumber': 5,
  'mediumDensity': 1000,
  'mediumViscosity': 0.0009,
  'vesselVolDay': 0.0,
  'perfLactateRate': 5.0,
  'perfAmmrate': 5.0,
  'growthRate': 0.029,
  'glutamineUptakeRate': 0,
  'glucoseUptakeRate': 0,
  'oxygenUptakeRate': 0.49,
  'carbonDioxideProdRate': 0.593,
  'ammoniaProductionRate': 0.013571,
  'lactateProductionRate': 0.135707,
  'massDensity': 1030,
  'cellRadius': 1.8e-05,
  'wetmass': 3000,
  'dryMassFraction': 0.3,
  'ammoniaLimit': 5,
  'lactateLimit': 50,
  'CO2Limit': 100,
  'turbLengthLimit': 2e-05,
  'count': 50,
  'doublings': 7,
  'rpmlims': (1, 75),
  'supervellims': (0.0001, 0.005),
  'celldenslims': (590000.0, 610000.0)},
 'outputs': {'overall': 47.63207161285757,
  'ammonia': 47.63207161285757,
  'lactate': 47.63207161285757,
  'CO2': 1

In [8]:
from pymongo import MongoClient

uri = "mongodb://localhost:27017/"
client = MongoClient(uri)

db = client['sims']
collection = db['table1']

In [9]:
def insertDf(kwargs, df):
    df_dict_list = df.to_dict(orient='records')
    df_dict = {}
    for item in df_dict_list:
        df_dict[item['Constraint']] = item['Maximum Yield [g/L wet]']
        
    document = {
        "inputs": {},
        "outputs": {}
    }
    
    for key, value in kwargs.items():
        document['inputs'][key] = value
    
    for key, value in df_dict.items():
        document['outputs'][key] = value

    existing_document = collection.find_one(document)
    if existing_document:
        print("A similar document already exists.")
    else:
        result = collection.insert_one(document)
        print(f"Document inserted with ID: {result.inserted_id}")

In [10]:
insertDf(kwargs, df)

Document inserted with ID: 658b57c6a46c86cdbd78789b


In [11]:
kwargs['temp'] = 100
df = runSim(**kwargs)
insertDf(kwargs, df)

Document inserted with ID: 658b57caa46c86cdbd78789c


In [12]:
#for i in range(100, 501, 25):
#    kwargs['temp'] = i
#    df = runSim(**kwargs)
#    insertDf(kwargs, df)


In [13]:
from pprint import pprint

cursor = db.table1.find()

# Iterate over the cursor and print each document
for document in cursor:
    pprint(document)

{'_id': ObjectId('658b57c6a46c86cdbd78789b'),
 'inputs': {'CO2Limit': 100,
            'ammoniaLimit': 5,
            'ammoniaProductionRate': 0.013571,
            'backPressure': 1.3,
            'carbonDioxideProdRate': 0.593,
            'cellRadius': 1.8e-05,
            'celldenslims': [590000.0, 610000.0],
            'count': 50,
            'doublings': 7,
            'dryMassFraction': 0.3,
            'glucoseUptakeRate': 0,
            'glutamineUptakeRate': 0,
            'growthRate': 0.029,
            'initCells': 4000000.0,
            'lactateLimit': 50,
            'lactateProductionRate': 0.135707,
            'massDensity': 1030,
            'mediumDensity': 1000,
            'mediumViscosity': 0.0009,
            'moleFracCO2': 0.03,
            'moleFracO2': 0.21,
            'oxygenUptakeRate': 0.49,
            'perfAmmrate': 5.0,
            'perfLactateRate': 5.0,
            'powerNumber': 5,
            'rpm': 42.3,
            'rpmlims': [1, 75],
         